# Классификациия комментариев для компании Самокат

Компания Самокат обратилась с задачей множественной классификации комментариев к заказам из приложения по заданным темам.
В задании представлены ответы на опрос, состоящий из части с выбором ответа из списка и расширенного комментария с произвольным текстом. Необходимо для каждого ответа из 50 различных меток классов выбрать все затронутые.

**Метрикой** в данной задаче является Accuracy, которая в множественной классификации считается через полное совпадение списка выбранных классов для каждого экземпляра. 

**Описание данных**:  
Исходные данные размещены в файле *train.csv*, тестовые данные в файле *test.csv*.  
Описание искомых категорий указано в файле *trends_description.csv*.  
Также есть пример отправки  результата в файле *sample_submission.csv* и базовая модель классиикации в файле *baseline.ipynb*. 

<a id='К-содержанию'></a>
**Содержание проекта** \
[Загрузка и предобработка данных](#Загрузка-и-предобработка-данных)\
[Подготовка и обучение моделей на векторах TF-IDF](#Подготовка-и-обучение-моделей-на-векторах-TF-IDF)\
[Подготовка и обучение моделей на эмбеддингах Word2Vec](#Подготовка-и-обучение-моделей-на-эмбеддингах-Word2Vec)\
[Подготовка и обучение моделей на эмбеддингах LaBSE](#Подготовка-и-обучение-моделей-на-эмбеддингах-LaBSE)\
[Проверка лучшей модели на валидационной выборке](#Проверка-лучшей-модели-на-валидационной-выборке)\
[Дообучение модели LaBSE для классификации](#Дообучение-модели-LaBSE-для-классификации)\
[Общий вывод](#Общий-вывод)

In [1]:
%%capture
!pip install spacy
!python -m spacy download ru_core_news_sm
!pip install gensim
!pip install sentence_transformers

In [2]:
#импортируем библиотеки перед началом проекта одним блоком
import os
import pandas as pd
import numpy as np
import random
import re
from tqdm.notebook import tqdm
import joblib


# загружаем класс pipeline и классы дя подготовки данных и запуска пайплайна
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from tqdm import notebook

# загружаем нужные модели и метрики
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import accuracy_score
from sklearn.base import BaseEstimator, ClassifierMixin

# загружаем функции для подготовки текстовых данных
import nltk
from nltk.tokenize import word_tokenize
import spacy
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer 

# загружаем библиотеки для Pytorch и предобученных моделей
import torch
from torch import nn
import torch.optim as optim
import torch.nn.functional as F
import transformers 
from sentence_transformers import SentenceTransformer, util
from transformers import AutoTokenizer, AutoModel, Trainer, TrainingArguments

In [3]:
# зададим стандарт датафрейма перед загрузкой
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

os.environ["TOKENIZERS_PARALLELISM"] = "false"

# устанавливаем фиксированное значение для всех случайных библиотек
RANDOM_STATE = 42

def seed_all(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.benchmark = True
        torch.backends.cudnn.deterministic = False
seed_all(RANDOM_STATE)

# функции для лемматизаци и исключения ненужных символов в комментариях
nlp = spacy.load("ru_core_news_sm")

def cleaner(text: str) -> str:
    
    text = re.sub(r'\_(,*?)\_', '', text)
    text = re.sub(r'[\n\t]', '', text)
    text = re.sub(r'<[^>]*>', '', text)
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)
    text = " ".join(text.split())
    return text

def lemmatize(text):
    doc = nlp(text)
    lemm_list = [token.lemma_ for token in doc if not token.is_punct] # сразу убираем пунктуацию 
    lemm_text = " ".join(lemm_list)
    return lemm_text

def process_text(text):
    text = cleaner(text)
    text = lemmatize(text)
    return text

<a id='Загрузка-и-предобработка-данных'></a> 
## Загрузка и предобработка данных
[К содержанию](#К-содержанию)

In [4]:
# импортируем тренировочные данные
pth1 = 'Data/train.csv'

if os.path.exists(pth1):
    data = pd.read_csv(pth1)
else:
    print('Something is wrong')

data.info()
display(data.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8708 entries, 0 to 8707
Data columns (total 54 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   index           8708 non-null   int64  
 1   assessment      8478 non-null   float64
 2   tags            8697 non-null   object 
 3   text            8708 non-null   object 
 4   trend_id_res0   8708 non-null   int64  
 5   trend_id_res1   8708 non-null   int64  
 6   trend_id_res2   8708 non-null   int64  
 7   trend_id_res3   8708 non-null   int64  
 8   trend_id_res4   8708 non-null   int64  
 9   trend_id_res5   8708 non-null   int64  
 10  trend_id_res6   8708 non-null   int64  
 11  trend_id_res7   8708 non-null   int64  
 12  trend_id_res8   8708 non-null   int64  
 13  trend_id_res9   8708 non-null   int64  
 14  trend_id_res10  8708 non-null   int64  
 15  trend_id_res11  8708 non-null   int64  
 16  trend_id_res12  8708 non-null   int64  
 17  trend_id_res13  8708 non-null   i

,index,assessment,tags,text,trend_id_res0,trend_id_res1,trend_id_res2,trend_id_res3,trend_id_res4,trend_id_res5,trend_id_res6,trend_id_res7,trend_id_res8,trend_id_res9,trend_id_res10,trend_id_res11,trend_id_res12,trend_id_res13,trend_id_res14,trend_id_res15,trend_id_res16,trend_id_res17,trend_id_res18,trend_id_res19,trend_id_res20,trend_id_res21,trend_id_res22,trend_id_res23,trend_id_res24,trend_id_res25,trend_id_res26,trend_id_res27,trend_id_res28,trend_id_res29,trend_id_res30,trend_id_res31,trend_id_res32,trend_id_res33,trend_id_res34,trend_id_res35,trend_id_res36,trend_id_res37,trend_id_res38,trend_id_res39,trend_id_res40,trend_id_res41,trend_id_res42,trend_id_res43,trend_id_res44,trend_id_res45,trend_id_res46,trend_id_res47,trend_id_res48,trend_id_res49
0,5652,6.0,"{ASSORTMENT,PROMOTIONS,DELIVERY}","Маленький выбор товаров, хотелось бы ассортимент больше, а так вроде бы все хорошо",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,18092,4.0,"{ASSORTMENT,PRICE,PRODUCTS_QUALITY,DELIVERY}",Быстро,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,13845,6.0,"{DELIVERY,PROMOTIONS,PRICE,ASSORTMENT,SUPPORT}",Доставка постоянно задерживается,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,25060,6.0,"{PRICE,PROMOTIONS,ASSORTMENT}",Наценка и ассортимент расстраивают,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,15237,5.0,"{ASSORTMENT,PRODUCTS_QUALITY,PROMOTIONS,CATALOG_NAVIGATION}",Доставка просто 👍,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [5]:
# проверим целевые столбцы на нулевые значения - нет смысла тренировать модель на пустых
target_columns = [f'trend_id_res{i}' for i in range(50)]
target_all_zeros = (data[target_columns] == 0).all(axis=1)
num_target_all_zeros = target_all_zeros.sum()

print(f"Количество строк с нулями в целевых столбцах: {num_target_all_zeros}")

Количество строк с нулями в целевых столбцах: 4085


С точки зрения обучения моделей данные строки не имеют смысла, однако, наборы текстов будут полезны для обучения эмбеддингов, поэтому оставим.

In [6]:
#отфильтруем и уберем строки, где все значения целевых столбцов равны 0
#data = data[~(data[target_columns] == 0).all(axis=1)]
#data = data.reset_index(drop=True)
#print("Строки с нулями удалены.")

In [7]:
# уберем столбец индекса отзыва
data = data.drop('index',axis=1)

In [8]:
# проверим дубликаты
print('Количество дубликатов: ', data.duplicated().sum().sum())

# проверим пропуски
print('Количество пропусков: ', data.isna().sum().sum())

# уберем дубликаты и пропуски
data = data.drop_duplicates()
data = data.dropna()

Количество дубликатов:  246
Количество пропусков:  241


In [9]:
# преобразуем значения категорий в столбце tags
data['tags'] = data['tags'].str.strip('{}').str.lower().str.split(',')
data.head()

,assessment,tags,text,trend_id_res0,trend_id_res1,trend_id_res2,trend_id_res3,trend_id_res4,trend_id_res5,trend_id_res6,trend_id_res7,trend_id_res8,trend_id_res9,trend_id_res10,trend_id_res11,trend_id_res12,trend_id_res13,trend_id_res14,trend_id_res15,trend_id_res16,trend_id_res17,trend_id_res18,trend_id_res19,trend_id_res20,trend_id_res21,trend_id_res22,trend_id_res23,trend_id_res24,trend_id_res25,trend_id_res26,trend_id_res27,trend_id_res28,trend_id_res29,trend_id_res30,trend_id_res31,trend_id_res32,trend_id_res33,trend_id_res34,trend_id_res35,trend_id_res36,trend_id_res37,trend_id_res38,trend_id_res39,trend_id_res40,trend_id_res41,trend_id_res42,trend_id_res43,trend_id_res44,trend_id_res45,trend_id_res46,trend_id_res47,trend_id_res48,trend_id_res49
0,6.0,"[assortment, promotions, delivery]","Маленький выбор товаров, хотелось бы ассортимент больше, а так вроде бы все хорошо",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,4.0,"[assortment, price, products_quality, delivery]",Быстро,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,6.0,"[delivery, promotions, price, assortment, support]",Доставка постоянно задерживается,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,6.0,"[price, promotions, assortment]",Наценка и ассортимент расстраивают,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,5.0,"[assortment, products_quality, promotions, catalog_navigation]",Доставка просто 👍,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [10]:
# собираем все категории в один список
all_categories = data['tags'].explode()

# смотрим уникальные значения категорий
all_categories.unique()

array(['assortment', 'promotions', 'delivery', 'price',
       'products_quality', 'support', 'catalog_navigation', 'payment'],
      dtype=object)

In [11]:
# категорий всего 8, можем закодировать их бинарно, создав 8 столбцов

for category in all_categories:
    data[f'tags_{category}'] = data['tags'].apply(lambda x: 1 if category in x else 0)

data = data.drop('tags', axis=1)

data.head()

,assessment,text,trend_id_res0,trend_id_res1,trend_id_res2,trend_id_res3,trend_id_res4,trend_id_res5,trend_id_res6,trend_id_res7,trend_id_res8,trend_id_res9,trend_id_res10,trend_id_res11,trend_id_res12,trend_id_res13,trend_id_res14,trend_id_res15,trend_id_res16,trend_id_res17,trend_id_res18,trend_id_res19,trend_id_res20,trend_id_res21,trend_id_res22,trend_id_res23,trend_id_res24,trend_id_res25,trend_id_res26,trend_id_res27,trend_id_res28,trend_id_res29,trend_id_res30,trend_id_res31,trend_id_res32,trend_id_res33,trend_id_res34,trend_id_res35,trend_id_res36,trend_id_res37,trend_id_res38,trend_id_res39,trend_id_res40,trend_id_res41,trend_id_res42,trend_id_res43,trend_id_res44,trend_id_res45,trend_id_res46,trend_id_res47,trend_id_res48,trend_id_res49,tags_assortment,tags_promotions,tags_delivery,tags_price,tags_products_quality,tags_support,tags_catalog_navigation,tags_payment
0,6.0,"Маленький выбор товаров, хотелось бы ассортимент больше, а так вроде бы все хорошо",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0
1,4.0,Быстро,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,0,0,0
2,6.0,Доставка постоянно задерживается,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,1,0,0
3,6.0,Наценка и ассортимент расстраивают,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0
4,5.0,Доставка просто 👍,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,1,0


In [12]:
print(f'Пример исходного текста тренировочной выборки: \n\n {data.iloc[0,1]} \n')

Пример исходного текста тренировочной выборки: 

 Маленький выбор товаров, хотелось бы ассортимент больше, а так вроде бы все хорошо 



In [13]:
# запускаем tqdm для отображения прогресса
tqdm.pandas()

data_lem = data.copy()

# выполняем лемматизацию с отображением прогресса
data_lem['text'] = data_lem['text'].progress_apply(lambda x: process_text(x))

# удалим пустые после лемматизации строки и соответствующие им значения целевого признака
data_lem = data_lem.dropna().reset_index(drop=True)

  0%|          | 0/8222 [00:00<?, ?it/s]

In [14]:
print(f'Пример преобразованного текста тренировочной выборки: \n\n {data_lem.iloc[0,1]} \n')

Пример преобразованного текста тренировочной выборки: 

 маленький выбор товар хотеться бы ассортимент большой а так вроде бы все хороший 



In [15]:
# добавим признак длины текста
data_lem['text_len'] = data_lem['text'].apply(lambda x: len(x.split()))

# добавим количества тегов
data_lem['tags_count'] = data_lem.filter(like='tags_').sum(axis=1)

data_lem.head()

,assessment,text,trend_id_res0,trend_id_res1,trend_id_res2,trend_id_res3,trend_id_res4,trend_id_res5,trend_id_res6,trend_id_res7,trend_id_res8,trend_id_res9,trend_id_res10,trend_id_res11,trend_id_res12,trend_id_res13,trend_id_res14,trend_id_res15,trend_id_res16,trend_id_res17,trend_id_res18,trend_id_res19,trend_id_res20,trend_id_res21,trend_id_res22,trend_id_res23,trend_id_res24,trend_id_res25,trend_id_res26,trend_id_res27,trend_id_res28,trend_id_res29,trend_id_res30,trend_id_res31,trend_id_res32,trend_id_res33,trend_id_res34,trend_id_res35,trend_id_res36,trend_id_res37,trend_id_res38,trend_id_res39,trend_id_res40,trend_id_res41,trend_id_res42,trend_id_res43,trend_id_res44,trend_id_res45,trend_id_res46,trend_id_res47,trend_id_res48,trend_id_res49,tags_assortment,tags_promotions,tags_delivery,tags_price,tags_products_quality,tags_support,tags_catalog_navigation,tags_payment,text_len,tags_count
0,6.0,маленький выбор товар хотеться бы ассортимент большой а так вроде бы все хороший,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,13,3
1,4.0,Быстро,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,0,0,0,1,4
2,6.0,доставка постоянно задерживаться,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,1,0,0,3,5
3,6.0,наценка и ассортимент расстраивать,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,4,3
4,5.0,доставка просто 👍,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,1,0,3,4


In [16]:
# сделаем индикатор "пустых" строк (где все значения равны 0)
is_empty = (data_lem[target_columns].sum(axis=1) == 0).astype(int)

# разделим выборку на тренировочную и тестовую с учетом стратификации по "пустым" признакам
X_train, X_test, y_train, y_test = train_test_split(
    data_lem.drop(target_columns,axis=1), data_lem[target_columns], test_size=0.15, random_state=RANDOM_STATE, stratify=is_empty)

X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

# проверим размерности после разбивки
print('Размерности датасетов после разбивки: \n',
    X_train.shape,
    X_test.shape,
    y_train.shape,
    y_test.shape)


Размерности датасетов после разбивки: 
 (6988, 12) (1234, 12) (6988, 50) (1234, 50)


In [17]:
# и преобразуем столбцы целевой переменной в массив numpy - для некоторых моделей критично
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()
display(y_train[:2],
        y_test[:2])

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0]])

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0]])

In [18]:
flag = 0

<a id='#Подготовка-и-обучение-моделей-на-векторах-TF-IDF'></a> 
## Подготовка и обучение моделей на векторах TF-IDF
[К содержанию](#К-содержанию)


Сначала попробуем самый простой вариант (на базе baseline модели) с применением векторов TF-IDF:

In [19]:
# закодируем комментарии с помощью tf-idf
X_train_tf_idf = X_train.copy()

# преобразуем тексты в tf-idf векторы
tf_idf_vectorizer = TfidfVectorizer() #analyzer="char_wb",ngram_range=(2,3))#,max_features=5000)
tf_idf_vectors = tf_idf_vectorizer.fit_transform(X_train_tf_idf['text'])

# преобразуем разреженную матрицу в DataFrame
tf_idf_df = pd.DataFrame(
    tf_idf_vectors.toarray(),
    columns=tf_idf_vectorizer.get_feature_names_out(),  # берем имена признаков
    index=X_train_tf_idf.index  # индексы для соответствия
)

# объединяем исходный датафрейм с признаками tf-idf
X_train_tf_idf = pd.concat([X_train_tf_idf, tf_idf_df], axis=1)

# удаляем ненужный столбец с текстами
X_train_tf_idf = X_train_tf_idf.drop(columns=['text'])

# выведем пару строк для проверки
X_train_tf_idf.head(2)

assessment  tags_assortment  tags_promotions  tags_delivery  tags_price  \
0         2.0                0                1              0           0   
1         6.0                1                1              1           0   

   tags_products_quality  tags_support  tags_catalog_navigation  tags_payment  \
0                      0             1                        0             0   
1                      0             1                        0             0   

   text_len  tags_count  always  cola  darkstore  \
0        11           2     0.0   0.0        0.0   
1        17           4     0.0   0.0        0.0   

   devochkavensdeigrustniemojitimnenravishsyatemchtoninakogonepohozha  done  \
0                                                                 0.0   0.0   
1                                                                 0.0   0.0   

   elit  fps  horosho  horror  huggies  jambo  klasss  knows  kruto  luchse  \
0   0.0  0.0      0.0     0.0      0.0    0.0     0.0    0.0    0.0     0.0   
1   0.0  0.0      0.0     0.0      0.0    0.0     0.0    0.0    0.0     0.0   

   mozhno  nestea   no       num  nuts  phone  pixel  pro  sale  soft  super  \
0     0.0     0.0  0.0  0.000000   0.0    0.0    0.0  0.0   0.0   0.0    0.0   
1     0.0     0.0  0.0  0.140885   0.0    0.0    0.0  0.0   0.0   0.0    0.0   

   top   up  vhs  well  who   wk  абонемент  абсолютно  абузить  авангард  \
0  0.0  0.0  0.0   0.0  0.0  0.0        0.0        0.0      0.0       0.0   
1  0.0  0.0  0.0   0.0  0.0  0.0        0.0        0.0      0.0       0.0   

   авка  авокадо  авто  автомат  автоматизация  автоматически   ад  адекват  \
0   0.0      0.0   0.0      0.0            0.0            0.0  0.0      0.0   
1   0.0      0.0   0.0      0.0            0.0            0.0  0.0      0.0   

   адекватно  адекватный  аджарски  администратор  адреналин  адрес  адресу  \
0        0.0         0.0       0.0            0.0        0.0    0.0     0.0   
1        0.0         0.0       0.0            0.0        0.0    0.0     0.0   

   адский   аж  аза  азбука  азбуке  азиатский  айс  айфон  айчары  академа  \
0     0.0  0.0  0.0     0.0     0.0        0.0  0.0    0.0     0.0      0.0   
1     0.0  0.0  0.0     0.0     0.0        0.0  0.0    0.0     0.0      0.0   

   аккаунт  аккуратно  аксессуар  активно  активность  актуальный  акцент  \
0      0.0        0.0        0.0      0.0         0.0         0.0     0.0   
1      0.0        0.0        0.0      0.0         0.0         0.0     0.0   

   акционные  акционных  акция  алко  алкоголь  алкогольный  альтернатива  \
0        0.0        0.0    0.0   0.0       0.0          0.0           0.0   
1        0.0        0.0    0.0   0.0       0.0          0.0           0.0   

   альтернативный  альтернативу  аля   ам  анализировать  аналог  аналогичный  \
0             0.0           0.0  0.0  0.0            0.0     0.0          0.0   
1             0.0           0.0  0.0  0.0            0.0     0.0          0.0   

   ананас  ангел  андроид  анигири  анус  апельсин  апельсинов  аподуктов  \
0     0.0    0.0      0.0      0.0   0.0       0.0         0.0        0.0   
1     0.0    0.0      0.0      0.0   0.0       0.0         0.0        0.0   

   аппетит  аптека  аргументировать  ареал  аромат  асортимент  асортиментов  \
0      0.0     0.0              0.0    0.0     0.0         0.0           0.0   
1      0.0     0.0              0.0    0.0     0.0         0.0           0.0   

   ассорти  ассортимент  ассортимента  ассортименту  ассорьимент  асфальт  \
0      0.0          0.0           0.0           0.0          0.0      0.0   
1      0.0          0.0           0.0           0.0          0.0      0.0   

   атас  атмосфера  аудитория  ахуели  ахуенно  ашан  аыышенные  бабка  \
0   0.0        0.0        0.0     0.0      0.0   0.0        0.0    0.0   
1   0.0        0.0        0.0     0.0      0.0   0.0        0.0    0.0   

   бабуля  бабушка  бабушкам  баг  багет  багуется  база  базовый  б

In [20]:
# посмотрим на размерность матрицы признаков
X_train_tf_idf.shape

(6988, 5509)

In [21]:
# создаём итоговый пайплайн с выбором модели и гиперпараметров для обучения на tf-idf
pipe_final_tfidf = Pipeline([
    ('models', MultiOutputClassifier(LogisticRegression(random_state=RANDOM_STATE)))
])

# Задаем наборы гиперпараметров для перебора
param_grid_tfidf = [
    # Словарь для модели LogisticRegression
    {
        'models': [MultiOutputClassifier(LogisticRegression(random_state=RANDOM_STATE))],
        'models__estimator__max_iter': [1000],
        'models__estimator__C': [10],
        'models__estimator__penalty': ['l2'],
        'models__estimator__class_weight': [None]
    },
    
    # Словарь для модели CatBoostClassifier
    {
        'models': [MultiOutputClassifier(CatBoostClassifier(silent=True, random_state=RANDOM_STATE, auto_class_weights='Balanced'))],
        'models__estimator__depth': [7],
        'models__estimator__iterations': [150],
        'models__estimator__learning_rate': [0.1],
        'models__estimator__l2_leaf_reg': [3]
    },
]

In [22]:
# полный перебор гиперпараметров с помощью GridSearchCV с применением tqdm

param_grid_tfidf = ParameterGrid(param_grid_tfidf)  # параметры для перебора внутри tqdm

if flag != 1:
    # настройка tqdm
    total_combinations = len(param_grid_tfidf)
    with tqdm(total=total_combinations) as pbar:
        best_score = -float('inf')
        best_params = None
        best_model = None


        for params in param_grid_tfidf:
            grid_search_tfidf = GridSearchCV(
                pipe_final_tfidf,
                {k: [v] for k, v in params.items()},  # подставляем конкретный набор параметров
                cv=5,
                scoring='accuracy',
                n_jobs=-1,
            )
            model_tfidf = grid_search_tfidf.fit(X_train_tf_idf, y_train)

            print(f'Метрика Accuracy: {model_tfidf.best_score_}')
            print(f'Параметры модели: {model_tfidf.best_params_}\n')

            # Обновление лучшей метрики
            if model_tfidf.best_score_ > best_score:
                best_score = model_tfidf.best_score_
                best_params = params
                best_model = model_tfidf

            pbar.update(1)  # обновляем прогресс бар tqdm

    # выгружаем лучшую метрику accuracy и параметры модели
    print(f'Лучшая метрика Accuracy: {best_score}')
    print(f'Параметры лучшей модели: {best_params}')


  0%|          | 0/2 [00:00<?, ?it/s]

Метрика Accuracy: 0.5113082089865572
Параметры модели: {'models': MultiOutputClassifier(estimator=LogisticRegression(random_state=42)), 'models__estimator__C': 10, 'models__estimator__class_weight': None, 'models__estimator__max_iter': 1000, 'models__estimator__penalty': 'l2'}

Метрика Accuracy: 0.5216117103582888
Параметры модели: {'models': MultiOutputClassifier(estimator=<catboost.core.CatBoostClassifier object at 0x336469c50>), 'models__estimator__depth': 7, 'models__estimator__iterations': 150, 'models__estimator__l2_leaf_reg': 3, 'models__estimator__learning_rate': 0.1}

Лучшая метрика Accuracy: 0.5216117103582888
Параметры лучшей модели: {'models': MultiOutputClassifier(estimator=<catboost.core.CatBoostClassifier object at 0x336469c50>), 'models__estimator__depth': 7, 'models__estimator__iterations': 150, 'models__estimator__l2_leaf_reg': 3, 'models__estimator__learning_rate': 0.1}


<a id='Подготовка-и-обучение-моделей-на-эмбеддингах-Word2Vec'></a> 
## Подготовка и обучение моделей на эмбеддингах Word2Vec
[К содержанию](#К-содержанию)

Далее попробуем усложнить процедуру подготовки текстовых данных - обучить Word2Vec на корпусе текстов, чтобы использовать эмбеддинги для сравнения текстов:

In [23]:
X_train_wv = X_train.copy()

# делаем токенизацию
X_train_wv['tokens'] = X_train_wv['text'].apply(lambda x: word_tokenize(x.lower()))
# тренируем Word2Vec только на тренировочной выборке
model_wv = Word2Vec(sentences=X_train_wv['tokens'], vector_size=150, window=5, min_count=1, workers=1, seed=RANDOM_STATE) 
# преобразуем текст в векторы
X_train_wv['text_wv'] = X_train_wv['tokens'].apply(lambda x: [model_wv.wv[word] for word in x if word in model_wv.wv])
# усредняем векторы и удаляем ненужные столбцы
X_train_wv['text_wv_avg'] = X_train_wv['text_wv'].apply(lambda x: np.mean(x, axis=0) if len(x) > 0 else np.zeros(model_wv.vector_size))
X_train_wv = X_train_wv.drop(columns=['text', 'tokens', 'text_wv'])


In [24]:
# развернем векторы в отдельные столбцы
vector_text = pd.DataFrame(X_train_wv['text_wv_avg'].tolist(), columns=[f'vector_{i}' for i in range(150)])
X_train_wv = pd.concat([X_train_wv.drop(columns=['text_wv_avg']), vector_text], axis=1)

print(X_train_wv.shape)

print(X_train_wv.index)
print(vector_text.index)

display(X_train_wv.shape)
display(X_train_wv.head())


(6988, 161)
RangeIndex(start=0, stop=6988, step=1)
RangeIndex(start=0, stop=6988, step=1)


(6988, 161)

,assessment,tags_assortment,tags_promotions,tags_delivery,tags_price,tags_products_quality,tags_support,tags_catalog_navigation,tags_payment,text_len,tags_count,vector_0,vector_1,vector_2,vector_3,vector_4,vector_5,vector_6,vector_7,vector_8,vector_9,vector_10,vector_11,vector_12,vector_13,vector_14,vector_15,vector_16,vector_17,vector_18,vector_19,vector_20,vector_21,vector_22,vector_23,vector_24,vector_25,vector_26,vector_27,vector_28,vector_29,vector_30,vector_31,vector_32,vector_33,vector_34,vector_35,vector_36,vector_37,vector_38,vector_39,vector_40,vector_41,vector_42,vector_43,vector_44,vector_45,vector_46,vector_47,vector_48,vector_49,vector_50,vector_51,vector_52,vector_53,vector_54,vector_55,vector_56,vector_57,vector_58,vector_59,vector_60,vector_61,vector_62,vector_63,vector_64,vector_65,vector_66,vector_67,vector_68,vector_69,vector_70,vector_71,vector_72,vector_73,vector_74,vector_75,vector_76,vector_77,vector_78,vector_79,vector_80,vector_81,vector_82,vector_83,vector_84,vector_85,vector_86,vector_87,vector_88,vector_89,vector_90,vector_91,vector_92,vector_93,vector_94,vector_95,vector_96,vector_97,vector_98,vector_99,vector_100,vector_101,vector_102,vector_103,vector_104,vector_105,vector_106,vector_107,vector_108,vector_109,vector_110,vector_111,vector_112,vector_113,vector_114,vector_115,vector_116,vector_117,vector_118,vector_119,vector_120,vector_121,vector_122,vector_123,vector_124,vector_125,vector_126,vector_127,vector_128,vector_129,vector_130,vector_131,vector_132,vector_133,vector_134,vector_135,vector_136,vector_137,vector_138,vector_139,vector_140,vector_141,vector_142,vector_143,vector_144,vector_145,vector_146,vector_147,vector_148,vector_149
0,2.0,0,1,0,0,0,1,0,0,11,2,0.269420,-0.698599,0.144264,-0.434142,0.005050,-0.049000,0.123394,0.356737,-0.109586,0.489625,-0.089212,0.173628,0.214612,0.107784,-0.254091,-0.110134,0.286879,-0.568271,-0.269523,-0.454514,0.114671,0.589652,-0.019308,0.541714,0.467113,-0.017316,-0.370478,-0.252914,0.227380,0.366879,0.234410,-0.707886,-0.383046,0.187913,-0.136802,0.155774,-0.023496,-0.720888,-0.133556,0.205031,-0.189871,0.142447,0.321754,-0.226063,0.222633,-0.155495,-0.153427,0.299812,-0.217065,-0.023675,-0.104360,-0.367099,0.048891,0.303340,0.435825,0.295652,0.209419,0.423931,0.041686,0.025207,-0.452143,0.028573,-0.316772,0.252120,-0.070870,0.012063,-0.090656,-0.433948,0.258720,0.588609,-0.330250,-0.468581,-0.184780,-0.344989,-0.456644,0.033412,0.233738,0.224216,-0.182102,-0.336585,0.288505,0.081539,0.081790,0.301899,-0.242400,-0.022471,-0.096310,-0.364742,0.005801,-0.238026,-0.368464,0.049245,-0.152618,-0.523735,-0.071146,-0.660275,0.177089,0.426944,0.787172,-0.316879,0.262455,0.010513,-0.304852,0.048048,-0.348359,-0.048663,-0.325810,-0.103200,-0.042355,0.054082,-0.070401,-0.064245,-0.273572,0.675380,0.381349,0.077370,0.295674,-0.079245,-0.654591,0.096651,0.275683,0.142818,0.461365,-0.457748,0.299060,-0.499147,0.198667,0.349230,-0.390367,-0.665117,-0.257060,-0.148384,0.353363,-0.221481,-0.368229,0.348700,0.147597,-0.355636,-0.100774,0.503864,-0.270609,0.298645,0.371153,-0.234142,0.203622,0.421260,0.205875,0.012797,-0.023719,-0.300668
1,6.0,1,1,1,0,0,1,0,0,17,4,0.243407,-0.713528,0.147451,-0.491553,-0.003491,-0.096329,0.119309,0.379456,-0.161355,0.510766,-0.089673,0.221333,0.274994,0.140361,-0.235619,-0.109876,0.305752,-0.575177,-0.284274,-0.462524,0.117464,0.630233,-0.011851,0.569802,0.512866,-0.005715,-0.340630,-0.255758,0.279953,0.358009,0.268182,-0.764257,-0.404378,0.200108,-0.112677,0.138314,-0.022025,-0.760402,-0.124583,0.184441,-0.217967,0.158817,0.386225,-0.225233,0.223768,-0.152070,-0.152805,0.316385,-0.226563,-0.013343,-0.099434,-0.398132,0.070225,0.319893,0.482762,0.309312,0.212264,0.416056,0.015565,0.073719,-0.416177,0.064900,-0.306816,0.244661,-0.031756,0.024343,-0.102768,-0.484589,0.281811,0.614022,-0.346357,-0.492586,-0.154381,-0.368699,-0.439181,0.044068,0.261058,0.215189,-0.167066,-0.399151,0.342781,0.059758,0.084667,0.326713,-0.234903,-0.019026,-0

In [25]:
# создаём итоговый пайплайн с выбором модели и гиперпараметров для обучения на эмбеддингах word2vec 
pipe_final_wv = Pipeline([
    ('models', MultiOutputClassifier(LogisticRegression(random_state=RANDOM_STATE)))
])

# задаем наборы гиперпараметров для перебора - используем аналогичные параметры, что и для tf-idf 
# оставляем лучшие наборы гиперпараметров, которые удалось получить на кросс-валидации - полный перебор опускаем

param_grid_wv = [
    # Словарь для модели LogisticRegression
    {
        'models': [MultiOutputClassifier(LogisticRegression(random_state=RANDOM_STATE))],
        'models__estimator__max_iter': [1000],
        'models__estimator__C': [10],
        'models__estimator__penalty': ['l2'],
        'models__estimator__class_weight': [None]
    },

    # Словарь для модели CatBoostClassifier
    {
        'models': [MultiOutputClassifier(CatBoostClassifier(silent=True, random_state=RANDOM_STATE, auto_class_weights='Balanced'))],
        'models__estimator__depth': [7],
        'models__estimator__iterations': [150],
        'models__estimator__learning_rate': [0.1],
        'models__estimator__l2_leaf_reg': [3],
    }
]

In [26]:
# полный перебор гиперпараметров с помощью GridSearchCV с применением tqdm
if flag != 1:
    param_grid_wv = ParameterGrid(param_grid_wv)  # параметры для перебора внутри tqdm

    # настройка tqdm
    total_combinations = len(param_grid_wv)
    with tqdm(total=total_combinations) as pbar:
        best_score = -float('inf')
        best_params = None
        best_model = None


        for params in param_grid_wv:
            grid_search_wv = GridSearchCV(
                pipe_final_wv,
                {k: [v] for k, v in params.items()},  # подставляем конкретный набор параметров
                cv=5,
                scoring='accuracy',
                n_jobs=-1,
            )
            model_wv = grid_search_wv.fit(X_train_wv, y_train)

            print(f'Метрика Accuracy: {model_wv.best_score_}')
            print(f'Параметры модели: {model_wv.best_params_}\n')

            # Обновление лучшей метрики
            if model_wv.best_score_ > best_score:
                best_score = model_wv.best_score_
                best_params = params
                best_model = model_wv

            pbar.update(1)  # обновляем прогресс бар tqdm

    # выгружаем лучшую метрику Accuracy и параметры модели
    print(f'Лучшая метрика Accuracy: {best_score}')
    print(f'Параметры лучшей модели: {best_params}')


  0%|          | 0/2 [00:00<?, ?it/s]

Метрика Accuracy: 0.48669251400149305
Параметры модели: {'models': MultiOutputClassifier(estimator=LogisticRegression(random_state=42)), 'models__estimator__C': 10, 'models__estimator__class_weight': None, 'models__estimator__max_iter': 1000, 'models__estimator__penalty': 'l2'}

Метрика Accuracy: 0.5635399993650813
Параметры модели: {'models': MultiOutputClassifier(estimator=<catboost.core.CatBoostClassifier object at 0x32e8e3650>), 'models__estimator__depth': 7, 'models__estimator__iterations': 150, 'models__estimator__l2_leaf_reg': 3, 'models__estimator__learning_rate': 0.1}

Лучшая метрика Accuracy: 0.5635399993650813
Параметры лучшей модели: {'models': MultiOutputClassifier(estimator=<catboost.core.CatBoostClassifier object at 0x32e8e3650>), 'models__estimator__depth': 7, 'models__estimator__iterations': 150, 'models__estimator__l2_leaf_reg': 3, 'models__estimator__learning_rate': 0.1}


<a id='Подготовка-и-обучение-моделей-на-эмбеддингах-LaBSE'></a> 
## Подготовка и обучение моделей на эмбеддингах LaBSE
[К содержанию](#К-содержанию)

Далее подключим предобученную модель LaBSE-ru-turbo (на базе BERT) для создания эмбеддингов предложений.

In [27]:
# загружаем модель
model_LaBSE = SentenceTransformer('sergeyzh/LaBSE-ru-turbo')

# копируем тренировочный датасет для возможной предобработки
X_train_bert = X_train.copy()

In [28]:
tqdm.pandas()

# преобразуем текст в векторы
X_train_bert['text_vect'] = X_train_bert['text'].progress_apply(lambda x: model_LaBSE.encode(x))

# удаляем ненужные столбцы
X_train_bert = X_train_bert.drop(columns=['text'])

X_train_bert.head(2)

  0%|          | 0/6988 [00:00<?, ?it/s]

,assessment,tags_assortment,tags_promotions,tags_delivery,tags_price,tags_products_quality,tags_support,tags_catalog_navigation,tags_payment,text_len,tags_count,text_vect
0,2.0,0,1,0,0,0,1,0,0,11,2,"[0.058266375, 0.0022367619, -0.02366651, 0.022975221, -0.033910956, 0.004358709, 0.0209202, -0.015890272, 0.022248752, 0.014778344, 0.015026953, -0.035889693, -0.0028999206, 0.013294978, 0.045170024, 0.007889779, 0.0004207931, 0.025048347, 0.014167981, -0.042518064, -0.03961556, -0.005228488, -0.0066043083, 0.036100764, 0.04221276, 0.025370171, -0.003304544, -0.0210686, -0.026388539, 0.03415403, -0.017136373, 0.006048383, 0.020480432, -0.019974388, -0.0010280486, -0.0104539525, 0.0035239344, 0.037149735, 0.04785923, -0.06885591, -0.01710669, -0.001799424, -0.0017648645, -0.01939534, -0.003730511, 0.016777463, -0.008671057, 0.006506621, -0.017093657, 0.017092971, -0.02702758, -0.00933273, 0.03854842, 0.006555776, 0.027438173, -0.0074756476, -0.02449172, 0.020876858, -0.004892325, -0.006262041, 0.0007645724, -0.020052182, 0.0031566955, 0.001788244, -0.0027312865, -0.03752136, -0.023035696, 0.020432653, 0.05881169, -0.022733416, -0.007107246, -0.029933728, -0.007101953, 0.001970953, -0.013242534, -0.02896522, 0.008165765, 0.023935718, -0.017479222, 0.034317687, -0.01183603, 0.032513697, 0.017316436, 0.028634626, -0.00062191376, 0.013601478, 0.013038712, -0.014413536, 0.037066307, 0.051547296, -0.0071508978, 0.025472954, -0.0113213565, -0.011264834, 0.049866345, 0.005914229, 0.043773346, -0.05123308, 0.0047616544, 0.02217594, ...]"
1,6.0,1,1,1,0,0,1,0,0,17,4,"[-0.010358731, -0.015012443, -0.029044122, 0.017889928, 0.007456112, 0.04176937, 0.03704001, 0.0022503205, -0.008878538, -0.009717544, 0.04383548, -0.0016043839, -0.007494684, 0.04988332, 0.020680266, 0.0018086971, 0.02961041, -0.05341848, 0.0054917857, 0.0365915, -0.04096404, -0.0014320223, -0.02387368, 0.004409981, 0.0070271245, 0.020686949, 0.007594833, 0.010309676, -0.017559292, 0.033824563, 0.013475971, 0.0397598, 0.042391844, -0.023852829, -0.016018573, 0.018561225, -0.0489381, 0.002632114, 0.01272903, -0.046140913, -0.019763567, 0.0055882838, -0.022248046, -0.0039494005, -0.07858931, -0.007885234, -1.0952115e-05, -0.007000204, 0.03041942, 0.0030753936, 0.033023827, 0.0023799024, 0.0031912313, -0.042145308, 0.06740845, 0.025509626, 0.0035841865, -0.032801915, -0.05313243, 0.036182154, 0.0010216872, 0.009137657, -0.009701383, 0.0026290906, 0.0045912405, 0.0037456844, -0.04884599, -0.0056053833, 0.052039117, -0.014190384, 0.0021712622, -0.01470924, -0.013629171, -0.008633817, -0.008837519, -0.033828594, -0.030072007, -0.025240628, -0.029948903, 0.021802751, -0.013357131, 0.023045307, -0.007136187, 0.047700435, 0.027580218, -0.0036972584, -0.02922318, 0.00499831, 0.010480978, 0.033894423, -0.015962956, 0.043963525, 0.006497946, 0.016390745, -0.02369114, 0.0011897342, -0.004474763, -0.013627285, 0.023259174, 0.018431773, ...]"


In [29]:
# определяем количество столбцов в векторе
vector_length = len(X_train_bert['text_vect'].iloc[0])

# генерируем названия столбцов
column_names = [f"text_vect_{i}" for i in range(vector_length)]

# создаем DataFrame с заданными названиями столбцов
text_vect_df = pd.DataFrame(X_train_bert['text_vect'].tolist(), 
                            index=X_train_bert.index, 
                            columns=column_names)

# объединяем с исходным DataFrame
X_train_bert = pd.concat([X_train_bert, text_vect_df], axis=1)

# удаляем колонку с исходным вектором
X_train_bert = X_train_bert.drop(columns=['text_vect'])

In [30]:
display(X_train_bert.head(2))
display(X_train_bert.shape)
y_train.shape

,assessment,tags_assortment,tags_promotions,tags_delivery,tags_price,tags_products_quality,tags_support,tags_catalog_navigation,tags_payment,text_len,tags_count,text_vect_0,text_vect_1,text_vect_2,text_vect_3,text_vect_4,text_vect_5,text_vect_6,text_vect_7,text_vect_8,text_vect_9,text_vect_10,text_vect_11,text_vect_12,text_vect_13,text_vect_14,text_vect_15,text_vect_16,text_vect_17,text_vect_18,text_vect_19,text_vect_20,text_vect_21,text_vect_22,text_vect_23,text_vect_24,text_vect_25,text_vect_26,text_vect_27,text_vect_28,text_vect_29,text_vect_30,text_vect_31,text_vect_32,text_vect_33,text_vect_34,text_vect_35,text_vect_36,text_vect_37,text_vect_38,text_vect_39,text_vect_40,text_vect_41,text_vect_42,text_vect_43,text_vect_44,text_vect_45,text_vect_46,text_vect_47,text_vect_48,text_vect_49,text_vect_50,text_vect_51,text_vect_52,text_vect_53,text_vect_54,text_vect_55,text_vect_56,text_vect_57,text_vect_58,text_vect_59,text_vect_60,text_vect_61,text_vect_62,text_vect_63,text_vect_64,text_vect_65,text_vect_66,text_vect_67,text_vect_68,text_vect_69,text_vect_70,text_vect_71,text_vect_72,text_vect_73,text_vect_74,text_vect_75,text_vect_76,text_vect_77,text_vect_78,text_vect_79,text_vect_80,text_vect_81,text_vect_82,text_vect_83,text_vect_84,text_vect_85,text_vect_86,text_vect_87,text_vect_88,text_vect_89,text_vect_90,text_vect_91,text_vect_92,text_vect_93,text_vect_94,text_vect_95,text_vect_96,text_vect_97,text_vect_98,text_vect_99,text_vect_100,text_vect_101,text_vect_102,text_vect_103,text_vect_104,text_vect_105,text_vect_106,text_vect_107,text_vect_108,text_vect_109,text_vect_110,text_vect_111,text_vect_112,text_vect_113,text_vect_114,text_vect_115,text_vect_116,text_vect_117,text_vect_118,text_vect_119,text_vect_120,text_vect_121,text_vect_122,text_vect_123,text_vect_124,text_vect_125,text_vect_126,text_vect_127,text_vect_128,text_vect_129,text_vect_130,text_vect_131,text_vect_132,text_vect_133,text_vect_134,text_vect_135,text_vect_136,text_vect_137,text_vect_138,text_vect_139,text_vect_140,text_vect_141,text_vect_142,text_vect_143,text_vect_144,text_vect_145,text_vect_146,text_vect_147,text_vect_148,text_vect_149,text_vect_150,text_vect_151,text_vect_152,text_vect_153,text_vect_154,text_vect_155,text_vect_156,text_vect_157,text_vect_158,text_vect_159,text_vect_160,text_vect_161,text_vect_162,text_vect_163,text_vect_164,text_vect_165,text_vect_166,text_vect_167,text_vect_168,text_vect_169,text_vect_170,text_vect_171,text_vect_172,text_vect_173,text_vect_174,text_vect_175,text_vect_176,text_vect_177,text_vect_178,text_vect_179,text_vect_180,text_vect_181,text_vect_182,text_vect_183,text_vect_184,text_vect_185,text_vect_186,text_vect_187,text_vect_188,text_vect_189,text_vect_190,text_vect_191,text_vect_192,text_vect_193,text_vect_194,text_vect_195,text_vect_196,text_vect_197,text_vect_198,text_vect_199,text_vect_200,text_vect_201,text_vect_202,text_vect_203,text_vect_204,text_vect_205,text_vect_206,text_vect_207,text_vect_208,text_vect_209,text_vect_210,text_vect_211,text_vect_212,text_vect_213,text_vect_214,text_vect_215,text_vect_216,text_vect_217,text_vect_218,text_vect_219,text_vect_220,text_vect_221,text_vect_222,text_vect_223,text_vect_224,text_vect_225,text_vect_226,text_vect_227,text_vect_228,text_vect_229,text_vect_230,text_vect_231,text_vect_232,text_vect_233,text_vect_234,text_vect_235,text_vect_236,text_vect_237,text_vect_238,text_vect_239,text_vect_240,text_vect_241,text_vect_242,text_vect_243,text_vect_244,text_vect_245,text_vect_246,text_vect_247,text_vect_248,text_vect_249,text_vect_250,text_vect_251,text_vect_252,text_vect_253,text_vect_254,text_vect_255,text_vect_256,text_vect_257,text_vect_258,text_vect_259,text_vect_260,text_vect_261,text_vect_262,text_vect_263,text_vect_264,text_vect_265,text_vect_266,text_vect_267,text_vect_268,text_vect_269,text_vect_270,text_vect_271,text_vect_272,text_vect_273,text_vect_274,text_vect_275,text_vect_276,text_vect_277,text_vect_278,text_vect_279,text_vect_280,text_vect_281,t

(6988, 779)

(6988, 50)

In [31]:
# создадим простую модель (MLP) на основе нейронной сети для обученяи на эмбеддингах LaBSE

# определяем MLP для многоклассовой задачи
class MLPModel(nn.Module):
    def __init__(self, input_dim, hidden_dims, output_dim):
        super(MLPModel, self).__init__()
        
        # задаем слои с разным количеством нейронов
        layers = []
        prev_dim = input_dim
        for hidden_dim in hidden_dims:
            layers.append(nn.Linear(prev_dim, hidden_dim))
            layers.append(nn.ReLU())
            prev_dim = hidden_dim
        self.hidden_layers = nn.Sequential(*layers)
        
        # выходной слой
        self.fc_out = nn.Linear(prev_dim, output_dim)
        
    def forward(self, x):
        x = self.hidden_layers(x)
        x = self.fc_out(x)
        return x

# классификатор с несколькими слоями
class MLPClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, input_dim, hidden_dims=[64, 128], output_dim=50, lr=0.001, epochs=10, batch_size=32):
        self.input_dim = input_dim
        self.hidden_dims = hidden_dims
        self.output_dim = output_dim
        self.lr = lr
        self.epochs = epochs
        self.batch_size = batch_size
        self.model = None
        self.classes_ = None  # Атрибут для хранения уникальных классов

    def _convert_to_numpy(self, X):
        """преобразуем входные данные в NumPy массив"""
        if isinstance(X, pd.DataFrame) or isinstance(X, pd.Series):
            return X.values
        elif isinstance(X, np.ndarray):
            return X
        else:
            raise ValueError("Неподдерживаемый формат данных. Необходимо передать DataFrame, Series, или массив NumPy.")

    def fit(self, X, y):
        X = self._convert_to_numpy(X)
        y = self._convert_to_numpy(y)

        X_tensor = torch.tensor(X, dtype=torch.float32)
        y_tensor = torch.tensor(y, dtype=torch.long)

        # создание модели
        self.model = MLPModel(self.input_dim, self.hidden_dims, self.output_dim)
        
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(self.model.parameters(), lr=self.lr)

        for epoch in range(self.epochs):
            self.model.train()
            epoch_loss = 0
            for i in range(0, len(X_tensor), self.batch_size):
                batch_X = X_tensor[i:i + self.batch_size]
                batch_y = y_tensor[i:i + self.batch_size]

                optimizer.zero_grad()
                outputs = self.model(batch_X)
                loss = criterion(outputs, batch_y)
                loss.backward()
                optimizer.step()

                epoch_loss += loss.item()  # накопление потерь для эпохи

            # печатаем потерю после обработки всех батчей в эпохе
            #print(f"Epoch {epoch + 1}/{self.epochs}, Loss: {epoch_loss / (len(X_tensor) // self.batch_size):.4f}")

        # сохраняем уникальные классы после обучения модели
        self.classes_ = np.unique(y)

        return self

    def predict_proba(self, X):
        X = self._convert_to_numpy(X)
        self.model.eval()
        X_tensor = torch.tensor(X, dtype=torch.float32)
        with torch.no_grad():
            outputs = self.model(X_tensor)
            probabilities = torch.softmax(outputs, dim=1).numpy()
        return probabilities

    def predict(self, X):
        probabilities = self.predict_proba(X)
        return np.argmax(probabilities, axis=1)

    def score(self, X, y):
        X = self._convert_to_numpy(X)
        y = self._convert_to_numpy(y)
        predictions = self.predict(X)
        return (predictions == y).mean()

In [32]:
# создаём итоговый пайплайн с выбором модели и гиперпараметров для обучения на эмбеддингах LaBSE
pipe_final_bert = Pipeline([
    ('models', MultiOutputClassifier(LogisticRegression(random_state=RANDOM_STATE)))
])

# задаем наборы гиперпараметров для перебора
# аналогично моделям выше - оставил только лучшие наборы по итогам кросс-валидации, полный перебор опускаем

param_grid_bert = [
    # Словарь для модели LogisticRegression
    {
        'models': [MultiOutputClassifier(LogisticRegression(random_state=RANDOM_STATE))],
        'models__estimator__max_iter': [1000],
        'models__estimator__C': [10],
        'models__estimator__penalty': ['l2'],
        'models__estimator__class_weight': [None]
    },

    # Словарь для модели CatBoostClassifier
    {
        'models': [MultiOutputClassifier(CatBoostClassifier(silent=True, random_state=RANDOM_STATE, auto_class_weights='Balanced',
                                                            early_stopping_rounds=10))],
        'models__estimator__depth': [7],
        'models__estimator__iterations': [400],
        'models__estimator__learning_rate': [0.1],
        'models__estimator__l2_leaf_reg': [3],
        'models__estimator__border_count': [128],
        'models__estimator__rsm': [1],
        'models__estimator__subsample': [1],
        'models__estimator__random_strength': [1],
        'models__estimator__bagging_temperature': [0]
    },

    # Словарь для модели MLPClassifier
    {
        'models': [MultiOutputClassifier(MLPClassifier(input_dim=X_train_bert.shape[1], output_dim=50,))],
        'models__estimator__hidden_dims': [[256, 128, 64]],
        'models__estimator__lr': [1],
        'models__estimator__epochs': [10],
        'models__estimator__batch_size': [256]    
    }
    ]

In [33]:

# полный перебор гиперпараметров с помощью GridSearchCV с применением tqdm
if flag != 1:
    param_grid_bert = ParameterGrid(param_grid_bert)  # параметры для перебора внутри tqdm

    # настройка tqdm
    total_combinations = len(param_grid_bert)
    with tqdm(total=total_combinations) as pbar:
        best_score_bert = -float('inf')
        best_params_bert = None
        best_model_bert = None

        for params in param_grid_bert:
            grid_search_bert = GridSearchCV(
                pipe_final_bert,
                {k: [v] for k, v in params.items()},  # подставляем конкретный набор параметров
                cv=5,
                scoring='accuracy',
                n_jobs=-1,
            )
            model_bert = grid_search_bert.fit(X_train_bert, y_train)

            print(f'Метрика Accuracy: {model_bert.best_score_}')
            print(f'Параметры модели: {model_bert.best_params_}\n')

            # обновляем метрику
            if model_bert.best_score_ > best_score_bert:
                best_score_bert = model_bert.best_score_
                best_params_bert = params
                best_model_bert = model_bert

            pbar.update(1)  # обновляем прогресс бар tqdm

    # выгружаем лучшую метрику Accuracy и параметры модели
    print(f'Лучшая метрика Accuracy: {best_score_bert}')
    print(f'Параметры лучшей модели: {best_params_bert}')

  0%|          | 0/3 [00:00<?, ?it/s]

Метрика Accuracy: 0.5306280677069093
Параметры модели: {'models': MultiOutputClassifier(estimator=LogisticRegression(random_state=42)), 'models__estimator__C': 10, 'models__estimator__class_weight': None, 'models__estimator__max_iter': 1000, 'models__estimator__penalty': 'l2'}

Метрика Accuracy: 0.6252189701414128
Параметры модели: {'models': MultiOutputClassifier(estimator=<catboost.core.CatBoostClassifier object at 0x32e95b9d0>), 'models__estimator__bagging_temperature': 0, 'models__estimator__border_count': 128, 'models__estimator__depth': 7, 'models__estimator__iterations': 400, 'models__estimator__l2_leaf_reg': 3, 'models__estimator__learning_rate': 0.1, 'models__estimator__random_strength': 1, 'models__estimator__rsm': 1, 'models__estimator__subsample': 1}

Метрика Accuracy: 0.44390503664607284
Параметры модели: {'models': MultiOutputClassifier(estimator=MLPClassifier(input_dim=779)), 'models__estimator__batch_size': 256, 'models__estimator__epochs': 10, 'models__estimator__hidde

In [34]:
# сохранение лучшей модели
#joblib.dump(best_model_bert, 'best_model_clear_bert.pkl')
#print("Модель сохранена в файл best_model_clear_bert.pkl")

<a id='Проверка-лучшей-модели-на-валидационной-выборке'></a> 
## Проверка лучшей модели на валидационной выборке
[К содержанию](#К-содержанию)

Выберем лучшую модель, про итогам кросс-валиаци на разныех вариантах преобразовния текстов (TF-IDF, Word2Vec, LaBSE) и проведем проверку на валидационной выборке, ранее выделенной из тренировочной и не участвовавшей в обучении. Лучшие результаты с метрикой accuracy на уровне 0.62 показала модель, обученная Catboost на эмбеддингах предобученной LaBSE, ее и будем тестировать:

In [35]:
X_test_bert = X_test.copy()

tqdm.pandas()

# преобразуем текст в векторы
X_test_bert['text_vect'] = X_test_bert['text'].progress_apply(lambda x: model_LaBSE.encode(x))

# удаляем ненужные столбцы
X_test_bert = X_test_bert.drop(columns=['text'])

X_test_bert.head(2)

  0%|          | 0/1234 [00:00<?, ?it/s]

,assessment,tags_assortment,tags_promotions,tags_delivery,tags_price,tags_products_quality,tags_support,tags_catalog_navigation,tags_payment,text_len,tags_count,text_vect
0,6.0,1,0,1,0,1,1,1,0,29,5,"[0.00036013086, -0.03528022, -0.014954457, -0.016956663, -0.038329218, 0.015523258, 0.0032164063, 0.006735514, 0.03324024, 0.00019466922, 0.011180547, -0.0044298964, 0.0058418955, -0.014067804, 0.028526459, -0.034279082, -0.01368375, -0.006148947, -0.023980413, 0.0072173835, -0.0017660378, 0.00023879904, -0.026463639, 0.0040453374, 0.020404646, 0.019818928, 0.0060669715, -0.011822134, -0.020403316, 0.003350783, -0.002955692, -0.0148564745, 0.02083568, -0.018679205, 0.019180125, -0.015484745, -0.012740105, -0.007253425, 0.0028211174, -0.08572251, -0.01269066, -0.004179113, -0.032811362, -0.017562827, -0.06885304, -0.021773728, -0.025296947, 0.0021211137, 0.012182426, 0.017812537, -0.045365468, -0.033700168, 0.00066258555, -0.03489903, 0.047786444, -0.014547288, 0.0038958837, -0.026518002, -0.058763005, 0.044459797, -0.020373352, 0.0017497065, 0.005072954, -0.0031589556, 0.028559007, -0.03427765, -0.0063339598, 0.02292376, 0.04019004, -0.008439728, -0.028162451, -0.010550224, -0.0087918565, -0.002249871, -0.003329785, -0.0016154009, 0.015256508, 0.01902247, -0.035366297, 0.0022000219, -0.02113619, 0.01716768, 0.013021038, 0.06080408, 0.01723275, 0.017695704, 0.005156194, -0.00017913237, 0.006565541, 0.0038728684, 0.004810301, -0.006106544, -0.021511126, 0.0647173, -0.003998983, -0.006218472, -0.03855435, -0.012560402, 0.016796058, 0.009137364, ...]"
1,5.0,1,1,1,1,1,1,1,1,8,8,"[0.0014719601, 0.003666016, 0.03653107, 0.012741787, -0.029573008, 0.04964126, 0.045388594, -0.0022070406, 0.02124903, 0.020281028, 0.03251101, -0.0060531427, -0.029702775, 0.012295499, 0.023246251, -0.029331433, -0.021245703, -0.072890855, -0.020343602, 0.0223762, -0.0005439848, -0.032734904, -0.005439613, -0.017454714, 0.029549953, 0.03015772, -0.023922894, 0.014462702, -0.023643294, -0.00030633475, 0.011224688, -0.03319629, 0.04012765, -0.025069052, -0.023498459, -0.00527323, -0.012405067, -0.009468706, -0.009173998, -0.028914195, 0.0068997606, -0.009306354, 0.029397132, -0.02375635, -0.044040058, -0.029295627, -0.0221922, -0.020300273, -0.021854483, 0.04127881, 0.0021152673, -0.00453095, 0.010675428, -0.017654605, 0.01910992, 0.021949073, 0.012644911, -0.03517846, -0.020413674, 0.015850566, -0.018897401, 0.030944655, 0.039899185, 0.012162827, 0.044273484, -0.03212384, -0.03885319, -0.008951, 0.051184177, -0.012568173, -0.013888352, -0.00475758, -0.015793577, -0.014115435, -0.010928495, -0.00089086784, 0.010601997, -0.0112208845, -0.011378393, 0.0060956483, -0.014545554, 0.040242795, 0.015106963, 0.033714097, 0.017067676, -0.04397802, -0.010570064, -0.010737927, 0.010406617, -0.013850631, -0.03139974, 0.001900732, 0.0076700244, 0.040339716, 0.021076638, -0.01857761, -0.049102925, -0.03662089, 0.019722639, -0.0058178203, ...]"


In [36]:
# определяем количество столбцов в векторе
vector_length = len(X_test_bert['text_vect'].iloc[0])

# генерируем названия столбцов
column_names = [f"text_vect_{i}" for i in range(vector_length)]

# создаем DataFrame с заданными названиями столбцов
text_vect_df = pd.DataFrame(X_test_bert['text_vect'].tolist(), 
                            index=X_test_bert.index, 
                            columns=column_names)

# объединяем с исходным DataFrame
X_test_bert = pd.concat([X_test_bert, text_vect_df], axis=1)

# удаляем колонку с исходным вектором
X_test_bert = X_test_bert.drop(columns=['text_vect'])

In [37]:
display(X_test_bert.head(2))
display(X_test_bert.shape)
y_test.shape

,assessment,tags_assortment,tags_promotions,tags_delivery,tags_price,tags_products_quality,tags_support,tags_catalog_navigation,tags_payment,text_len,tags_count,text_vect_0,text_vect_1,text_vect_2,text_vect_3,text_vect_4,text_vect_5,text_vect_6,text_vect_7,text_vect_8,text_vect_9,text_vect_10,text_vect_11,text_vect_12,text_vect_13,text_vect_14,text_vect_15,text_vect_16,text_vect_17,text_vect_18,text_vect_19,text_vect_20,text_vect_21,text_vect_22,text_vect_23,text_vect_24,text_vect_25,text_vect_26,text_vect_27,text_vect_28,text_vect_29,text_vect_30,text_vect_31,text_vect_32,text_vect_33,text_vect_34,text_vect_35,text_vect_36,text_vect_37,text_vect_38,text_vect_39,text_vect_40,text_vect_41,text_vect_42,text_vect_43,text_vect_44,text_vect_45,text_vect_46,text_vect_47,text_vect_48,text_vect_49,text_vect_50,text_vect_51,text_vect_52,text_vect_53,text_vect_54,text_vect_55,text_vect_56,text_vect_57,text_vect_58,text_vect_59,text_vect_60,text_vect_61,text_vect_62,text_vect_63,text_vect_64,text_vect_65,text_vect_66,text_vect_67,text_vect_68,text_vect_69,text_vect_70,text_vect_71,text_vect_72,text_vect_73,text_vect_74,text_vect_75,text_vect_76,text_vect_77,text_vect_78,text_vect_79,text_vect_80,text_vect_81,text_vect_82,text_vect_83,text_vect_84,text_vect_85,text_vect_86,text_vect_87,text_vect_88,text_vect_89,text_vect_90,text_vect_91,text_vect_92,text_vect_93,text_vect_94,text_vect_95,text_vect_96,text_vect_97,text_vect_98,text_vect_99,text_vect_100,text_vect_101,text_vect_102,text_vect_103,text_vect_104,text_vect_105,text_vect_106,text_vect_107,text_vect_108,text_vect_109,text_vect_110,text_vect_111,text_vect_112,text_vect_113,text_vect_114,text_vect_115,text_vect_116,text_vect_117,text_vect_118,text_vect_119,text_vect_120,text_vect_121,text_vect_122,text_vect_123,text_vect_124,text_vect_125,text_vect_126,text_vect_127,text_vect_128,text_vect_129,text_vect_130,text_vect_131,text_vect_132,text_vect_133,text_vect_134,text_vect_135,text_vect_136,text_vect_137,text_vect_138,text_vect_139,text_vect_140,text_vect_141,text_vect_142,text_vect_143,text_vect_144,text_vect_145,text_vect_146,text_vect_147,text_vect_148,text_vect_149,text_vect_150,text_vect_151,text_vect_152,text_vect_153,text_vect_154,text_vect_155,text_vect_156,text_vect_157,text_vect_158,text_vect_159,text_vect_160,text_vect_161,text_vect_162,text_vect_163,text_vect_164,text_vect_165,text_vect_166,text_vect_167,text_vect_168,text_vect_169,text_vect_170,text_vect_171,text_vect_172,text_vect_173,text_vect_174,text_vect_175,text_vect_176,text_vect_177,text_vect_178,text_vect_179,text_vect_180,text_vect_181,text_vect_182,text_vect_183,text_vect_184,text_vect_185,text_vect_186,text_vect_187,text_vect_188,text_vect_189,text_vect_190,text_vect_191,text_vect_192,text_vect_193,text_vect_194,text_vect_195,text_vect_196,text_vect_197,text_vect_198,text_vect_199,text_vect_200,text_vect_201,text_vect_202,text_vect_203,text_vect_204,text_vect_205,text_vect_206,text_vect_207,text_vect_208,text_vect_209,text_vect_210,text_vect_211,text_vect_212,text_vect_213,text_vect_214,text_vect_215,text_vect_216,text_vect_217,text_vect_218,text_vect_219,text_vect_220,text_vect_221,text_vect_222,text_vect_223,text_vect_224,text_vect_225,text_vect_226,text_vect_227,text_vect_228,text_vect_229,text_vect_230,text_vect_231,text_vect_232,text_vect_233,text_vect_234,text_vect_235,text_vect_236,text_vect_237,text_vect_238,text_vect_239,text_vect_240,text_vect_241,text_vect_242,text_vect_243,text_vect_244,text_vect_245,text_vect_246,text_vect_247,text_vect_248,text_vect_249,text_vect_250,text_vect_251,text_vect_252,text_vect_253,text_vect_254,text_vect_255,text_vect_256,text_vect_257,text_vect_258,text_vect_259,text_vect_260,text_vect_261,text_vect_262,text_vect_263,text_vect_264,text_vect_265,text_vect_266,text_vect_267,text_vect_268,text_vect_269,text_vect_270,text_vect_271,text_vect_272,text_vect_273,text_vect_274,text_vect_275,text_vect_276,text_vect_277,text_vect_278,text_vect_279,text_vect_280,text_vect_281,t

(1234, 779)

(1234, 50)

In [38]:
# делаем прогнозы на основе тестовых данных
X_test_pred = best_model_bert.predict(X_test_bert)

# считаем Accuracy для многометочной классификации
accuracy = accuracy_score(y_test, X_test_pred)

print('Метрика Accuracy на тестовых данных:', accuracy)

Метрика Accuracy на тестовых данных: 0.6523500810372771


<a id='Дообучение-модели-LaBSE-для-классификации'></a> 
## Дообучение модели LaBSE для классификации
[К содержанию](#К-содержанию)

Дополнительно попробуем проверить гипотезу с дообучением слоя классификации поверх самой модели LaBSE.

In [39]:
# скопируем тренировочный датасет для обработки под модель в случае необходимости
X_train_LaBSE_ft = X_train.copy()

In [40]:
# данная модель принимает предложения длиной, не больше 512 токенов, поэтому сначала проверю, какая максимальная длина в train и test:

seq_len_train = [len(str(i).split()) for i in X_train_LaBSE_ft['text']]
seq_len_test = [len(str(i).split()) for i in X_test['text']]
max_seq_len = max(max(seq_len_test), max(seq_len_train))
max_seq_len

52

In [41]:
# загружаем токенизатор и исходную модель LaBSE
tokenizer = AutoTokenizer.from_pretrained('sergeyzh/LaBSE-ru-turbo')
model_LaBSE_ft = AutoModel.from_pretrained('sergeyzh/LaBSE-ru-turbo')

In [42]:
# создаем модель с добавлением классификационного слоя

class LaBSEClassifier(nn.Module):
    def __init__(self, base_model, num_labels):
        super().__init__()
        self.base_model = base_model  # исходная модель
        self.output_layer = nn.Linear(768, num_labels)  # слой классификатора

    def forward(self, input_ids, attention_mask, labels=None):
        # получаем выходы из исходной модели
        outputs = self.base_model(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0]  # Используем выход [CLS] токена

        # пропускаем через классификационный слой
        logits = self.output_layer(pooled_output)

        # вычисляем функцию потерь
        loss = None
        if labels is not None:
            loss = F.binary_cross_entropy_with_logits(logits, labels)

        return {"loss": loss, "logits": logits} if loss is not None else logits
    
class TextDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = np.array(labels, dtype=np.float32)  # Используем float32 для работы с BCEWithLogitsLoss

    def __getitem__(self, idx):
        item = {
            'input_ids': self.encodings['input_ids'][idx].clone().detach(),
            'attention_mask': self.encodings['attention_mask'][idx].clone().detach(),
            'labels': torch.tensor(self.labels[idx], dtype=torch.float32)  # Метки в формате [batch_size, num_labels]
        }
        return item

    def __len__(self):
        return len(self.labels)

# функция для подготовки,токенизации данных
def prepare_data(texts, labels, tokenizer, test_size=0.2):
    
    assert len(texts) == len(labels), "Количество текстов и меток должно совпадать"
    
    texts = [str(text) for text in texts]
    labels = np.array(labels, dtype=np.float32)
    
    train_texts, val_texts, train_labels, val_labels = train_test_split(
        texts, labels, test_size=test_size, random_state=42, shuffle=True
    )
    
    train_encodings = tokenizer(
        train_texts,
        truncation=True,
        padding=True,
        max_length=512,
        return_tensors="pt"
    )
    
    val_encodings = tokenizer(
        val_texts,
        truncation=True,
        padding=True,
        max_length=512,
        return_tensors="pt"
    )
    
    train_dataset = TextDataset(train_encodings, train_labels)
    val_dataset = TextDataset(val_encodings, val_labels)
    
    return train_dataset, val_dataset

# функция расчета метрик
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    
    # преобразуем логиты в вероятности
    probabilities = torch.sigmoid(torch.tensor(logits)).numpy()
    # а потом в бинарные метки с порогом 0.5
    predictions = (probabilities > 0.5).astype(int)
    
    # считаем метрики для многозадачного случая
    accuracy = (predictions == labels).mean()  # Доля правильных предсказаний (по всем задачам)
    
    return {"accuracy": accuracy}


In [43]:
# инициализируем модель
classifier_LaBSE_ft = LaBSEClassifier(model_LaBSE_ft, num_labels=50)

# готовим данные
train_LaBSE_ft, val_LaBSE_ft = prepare_data(X_train_LaBSE_ft['text'], y_train, tokenizer)

# задаем параметры дообучения
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_strategy='epoch',
    logging_strategy='epoch',
    save_strategy='epoch',
    logging_dir='./logs',
    load_best_model_at_end=True,
    metric_for_best_model='loss',
    greater_is_better=False,
    warmup_steps=100,
    weight_decay=0.01,
    learning_rate=1e-6
)

# инициализируем тренера
trainer = Trainer(
    model=classifier_LaBSE_ft,
    args=training_args,
    train_dataset=train_LaBSE_ft,
    eval_dataset=val_LaBSE_ft,
    compute_metrics=compute_metrics
)


In [44]:
# запускаем дообучение модели
if flag != 1:
    trainer.train()

    # выводим метрики
    print(trainer.evaluate())  

  0%|          | 0/2796 [00:00<?, ?it/s]

{'loss': 0.2843, 'grad_norm': 0.39627620577812195, 'learning_rate': 7.778189910979228e-07, 'epoch': 1.0}


  0%|          | 0/175 [00:00<?, ?it/s]

{'eval_loss': 0.12507055699825287, 'eval_accuracy': 0.9833905579399141, 'eval_runtime': 13.1778, 'eval_samples_per_second': 106.087, 'eval_steps_per_second': 13.28, 'epoch': 1.0}
{'loss': 0.1195, 'grad_norm': 0.3482030928134918, 'learning_rate': 5.185459940652819e-07, 'epoch': 2.0}


  0%|          | 0/175 [00:00<?, ?it/s]

...

{'eval_loss': 0.10168487578630447, 'eval_accuracy': 0.9833905579399141, 'eval_runtime': 14.0307, 'eval_samples_per_second': 99.638, 'eval_steps_per_second': 12.473, 'epoch': 3.0}
{'loss': 0.1035, 'grad_norm': 0.2780798673629761, 'learning_rate': 0.0, 'epoch': 4.0}


  0%|          | 0/175 [00:00<?, ?it/s]

{'eval_loss': 0.09973445534706116, 'eval_accuracy': 0.9833905579399141, 'eval_runtime': 15.0296, 'eval_samples_per_second': 93.016, 'eval_steps_per_second': 11.644, 'epoch': 4.0}
{'train_runtime': 1134.6937, 'train_samples_per_second': 19.706, 'train_steps_per_second': 2.464, 'train_loss': 0.1537844749308792, 'epoch': 4.0}


  0%|          | 0/175 [00:00<?, ?it/s]

{'eval_loss': 0.09973445534706116, 'eval_accuracy': 0.9833905579399141, 'eval_runtime': 15.9149, 'eval_samples_per_second': 87.842, 'eval_steps_per_second': 10.996, 'epoch': 4.0}


Поскольку кросс-валидация не проводилась при дообучении - проведем оценку на тестовой выборке, подготовим валидационные данные для тестирования дообученной модели.

In [45]:
# выбираем устройство (MPS, если доступен)
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")

# перемещаем модель на устройство
classifier_LaBSE_ft.to(device)

def predict_on_test(model, texts):
    model.eval()  # включаем режим оценки
    
    texts = [str(text) for text in texts]
    
    # делаем токенизацию входных текстов
    encodings = tokenizer(
        texts,
        truncation=True,
        padding=True,
        max_length=512,
        return_tensors="pt"
    )
    
    # перемещаем данных на устройство
    input_ids = encodings['input_ids'].to(device)
    attention_mask = encodings['attention_mask'].to(device)
    
    # получаем предсказания
    with torch.no_grad():
        logits = model(input_ids=input_ids, attention_mask=attention_mask)
    
    # преобразуем полученные логиты в метки с тем же порогом
    predictions = (torch.sigmoid(logits) > 0.5).long()
    return predictions.cpu().numpy()

In [46]:
# делаем прдсказания на валидационном наборе данных
X_test_pred = predict_on_test(classifier_LaBSE_ft, X_test['text'])

# считаем Accuracy для многометочной классификации
accuracy = accuracy_score(y_test, X_test_pred)

print('Метрика Accuracy на тестовых данных:', accuracy)

Метрика Accuracy на тестовых данных: 0.44408427876823336


<a id='Общий-вывод'></a> 
## Общий вывод
[К содержанию](#К-содержанию)


По итогу **загрузки данных** было определено, что тренировочный датасет данных невелик - 8708 строк, при этом наличествует порядка 250 пропусков и дубликатов, которые были сразу исключены. Также выявлено, что почти половина датасета не имеет признаков принадлежности ни к одному из целевых классов (все значения из 50 целевых классов нулевые) - исключать данные строки не стали для улучшения обучения эмбеддингов (т.к. некоторая часть срок действительно могла не принадлежать ни к одному из классов).  
Далее была проведена незначительная **предобработка данных**:  
- значения столбца tags были бинарно закодированы в 8 столбцов(8 уникальных значений);
- проведена лемматизация и очистка текстов комментариев от пунктуационных и служебных знаков;
- добавлены признаки длины комментария и количества тегов.
После этого данные были разделены на тренировочную и тестовую выборки с учетом стратификаци по "пустым" целевым значениям (см. выше).  

**Исследовательский анализ** данных не проводился, т.к. основным источником для классификации в искомых признаков являются тексты самих комментариев, единственный числовой признак - оценка доставки имеет сильно второстепенное значение для классификации комментария по одной из 50 категорий.  

Далее были сформированы несколько пайплайнов с нарастанием сложности обучаемых моделей:  

1. Сначала текст коментариев были векторизованы с помощью метода **TF-IDF**, на этих векторах, а также прочих исходных и добавленных признаках были обучены две модели с помощью базового алгоритма логистической регрессии (метрика accuracy на кросс-валидации составила 0.51) и более продвинутого алгоритма catboost (метрика accuracy - 0.52), обе модели показали минимальные значения на уровне baseline'а и совсем незначительно превысили случайный вариант распределения меток классов.
2. Далее на тренировочном датасете была обучена модель подготовки **эмбеддингов Word2Vec**, с помощью которой были сгенерированы эмбеддинги, на которых (а также прочих признаках) были обучены аналогичные модели с помощью логистической регрессии (метрика accuracy составила 0.48) и catboost (метрика accuracy - 0.56), эмбеддинги word2vec лучше представили связи в текстах, что помогло catboost качественне выявить закономерности для итоговой классификации.
3. Следующим этапом была использована **предобученная модель генерации эмбеддингов LaBSE** (на базе BERT), на эмбеддингах которой были обучены 3 типа моделей - с помощью логистической регрессии (метрика accuracy - 0.53), catboost (метрика accuracy - 0.62) и нейронная сеть MLP с тремя полносвязными слоями (метрика accuracy - 0.44). В данном случае модель обученная catboost значительно превзошла другие варианты.
4. Финальным этапом было проведено **дообучение исходной модели LaBSE** для классификации 50 целевых признаков (с добавлением поверх предобученной модели классификационного слоя), данная модель показала также низкий результат по метрике accuracy на уровне 0.44.  
   
Для проверки **на тестовом наборе** была отобрана **модель обученная catboost'ом на эмбеддингах LaBSE**, которая показала метрику **accuracy - 0.65**.

В заверешение анализа можно предположить, что улучшение метрики относительно baseline'a c 0.50 до 0.65 с учетом настолько незначительного датасета (6 тысяч строк, из которых половина с нулевыми целевыми классами, то есть фактически для обучения классификации по 50! классам использовались всего 3 тысячи строк) является неплохим результатом, а в целях улучшения результата можно рекомендовать:
- увеличить обучающую выборку;
- усовершенствовать нейронные сети для обучения моделей с помощью архитетур c памятью контекста (GRU/LSTM), а также специализированных архитектур для табличных данных (TabM);
- проанализировать и дополнить признаковое пространство - учитывая частоту ключевых слов в текстах, предварительной классификации "пустых" признаков с помощью предобученных моделей, также учета норы эмбеддингов, наличия/частоты специальных слов/эмодзи и пр.